In [1]:
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def resize_video(video_path, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return None, None

    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, target_size)
        frames.append(resized_frame)

    cap.release()
    return frames, fps

In [3]:
def print_video_metadata(video_path):
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        print(f"Error opening video file: {video_path}")
        return None, None
   
    # Get the framerate
    fr = vid.get(cv2.CAP_PROP_FPS)

    # Total number of frames
    nframes = vid.get(cv2.CAP_PROP_FRAME_COUNT)

    # Duration
    dur = nframes/fr
    
    # Video height and width
    height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
    
    print(f"{os.path.basename(video_path)}, Frame rate: {int(fr)}, No. of Frames : {int(nframes)}, Duration:{dur:0.2f}s, Height {height}, Width {width}")

    vid.release()

In [4]:
def normalize_fps(frames, fps, target_fps=30):
    if fps == target_fps:
        return frames

    num_frames = len(frames)
    duration = num_frames / fps
    target_num_frames = int(duration * target_fps)

    frame_indices = np.linspace(0, num_frames - 1, target_num_frames, dtype=int)
    normalized_frames = [frames[i] for i in frame_indices]
    return normalized_frames

In [5]:
def apply_brightness_normalization(frames):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    normalized_frames = []
    for frame in frames:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cl = clahe.apply(gray)
        frame[:, :, 0] = cl
        frame[:, :, 1] = cl
        frame[:, :, 2] = cl
        normalized_frames.append(frame)
    return normalized_frames

In [6]:
def calculate_frame_difference(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(gray1, gray2)
    _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
    return np.sum(thresh)


def select_motion_videos(frames, motion_threshold=50000):
    total_motion = 0
    for i in range(len(frames) - 1):
        total_motion += calculate_frame_difference(frames[i], frames[i + 1])
    return total_motion > motion_threshold

In [7]:
def normalize_video_duration(frames, target_length=30):
    num_frames = len(frames)
    if num_frames == target_length:
        return frames
    elif num_frames < target_length:
        frame_indices = np.linspace(0, num_frames - 1, target_length, dtype=int)
        interpolated_frames = [frames[i] for i in frame_indices]
        return interpolated_frames
    else:
        frame_indices = np.linspace(0, num_frames - 1, target_length, dtype=int)
        sampled_frames = [frames[i] for i in frame_indices]
        return sampled_frames

In [15]:
def save_video(frames, output_path, target_fps=30):
    height, width, _ = frames[0].shape
    #fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fourcc = cv2.VideoWriter_fourcc(*"avc1")
    #fourcc = -1
    #fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    try:
        out = cv2.VideoWriter(output_path, fourcc, target_fps, (width, height))
    except Exception as e:
        print(f"Writing error: {e}")
   
    for frame in frames:
        out.write(frame)
    out.release()


def preprocess_pipeline(
    input_root,
    output_root,
    #target_size=(224, 224),
    target_size=(270, 270),
    target_fps=30,
    target_length=30
):
    

    # Iterate over subfolders corresponding to labels
    for label in os.listdir(input_root):
        label_input_folder = os.path.join(input_root, label)
        label_output_folder = os.path.join(output_root, label)

        if not os.path.isdir(label_input_folder):  # Skip non-folder entries
            continue

        os.makedirs(label_output_folder, exist_ok=True)
        print(f"Processing {label} Videos")
        
        # Iterate over video files in the label subfolder
        for file in os.listdir(label_input_folder):
            
            video_path = os.path.join(label_input_folder, file)
            output_path = os.path.join(
                label_output_folder,
                f"{os.path.splitext(file)[0]}_processed.mp4")
            #print(output_path)

            try:
                # collect metadata from video files
                #print_video_metadata(video_path)
                
                # Resize video
                frames, fps = resize_video(video_path, target_size)
                if frames is None:
                    continue

                # Normalize FPS
                frames = normalize_fps(frames, fps, target_fps)

                # Brightness normalization
                #frames = apply_brightness_normalization(frames)

                # Motion-based selection
                # if not select_motion_videos(frames, motion_threshold):
                #     continue

                # Normalize duration
                frames = normalize_video_duration(frames, target_length)
                #print(len(frames))

                # Save the final processed video
                save_video(frames, output_path, target_fps)

            except Exception as e:
                print(f"Error processing {video_path}: {e}")

In [17]:
input_folder = "/Users/savin/Omdena-Projects/Indonesia-SLR/data/raw_data/"  # Path to input videos
output_folder = "/Users/savin/Omdena-Projects/Indonesia-SLR/data/data_processed_v1/"  # Path to save final processed videos
preprocess_pipeline(input_folder, output_folder)

Processing lihat Videos


OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write vide

Processing ibu Videos


OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
2024-12-29 22:31:27.714 python[2392:41423] WARNING: -finishWriting should not be called on the main thread.
OpenCV: AV

Processing selamat Videos


...

OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
2024-12-29 22:32:27.921 python[2392:41423] WARNING: -finishWriting should not be called on the main thread.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AV

Processing senyum Videos


OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
2024-12-29 22:32:29.933 python[2392:41423] WARNING: -finishWriting should no

Processing buka Videos


OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
OpenCV: AVF: waiting to write video data.
2024-12-29 22:32:31.939 python[2392:41423] WARNING: -finishWriting should not be called on the main thread.
OpenCV: AV